<a href="https://colab.research.google.com/github/fani-lab/OpeNTF/blob/main/ipynb/tntf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center"><img src='https://raw.githubusercontent.com/fani-lab/OpeNTF/refs/heads/main/docs/figs/temporal.jpg' width="500" ></p>

`OpeNTF` via `Temporal (Streaming)` Training Strategy

Inspired by [`curriculum learning`](https://doi.org/10.1109/TPAMI.2021.3069908) and [`temporal latent space`](https://doi.org/10.1109/TKDE.2016.2591009) inference, we developed a temporal training strategy to consume time information as an aspect of experts' skills and collaboration ties in teams, as opposed to an extra numeric input feature, while being model-agnostic and with `no modification` to the models' architectures. In contrast to the `i.i.d` assumption (`bag of teams`) during model training on a shuffled dataset, we organize the teams based on time intervals (e.g., `yearly`) and train a neural model incrementally on streamed subsets of teams, as shown above. We randomly initialize a model's parameters at `t=0` and train the model on the subset of teams in the first time interval for a certain number of epochs. We then continue training on the subsequent time intervals, using the learned parameters from the previous time interval. This process is repeated until we complete the training on the subset of teams in the last time interval. Via temporal training, models capture the evolution of experts' skills and collaborative ties over time.



---



**Temporal (Streaming) Team Recommendation [`[ECIR24]`](https://doi.org/10.1007/978-3-031-56027-9\_20)**

To run `OpeNTF` in temporal mode, the model instance should be set to `mdl.tntf.tNtf_{mdl.rnd.Rnd,mdl.fnn.Fnn,mdl.bnn.Bnn}` in [`src/__config__.yaml#L65`](https://github.com/fani-lab/OpeNTF/blob/main/src/__config__.yaml#L65) like:

Random model: `"models.instances=[mdl.tntf.tNtf_mdl.rnd.Rnd]"`

Non-variational feedforward model: `"models.instances=[mdl.tntf.tNtf_mdl.fnn.Fnn]"`

Variational Bayesian model: `"models.instances=[mdl.tntf.tNtf_mdl.bnn.Bnn]"`

All models: `"models.instances=[mdl.tntf.tNtf_mdl.rnd.Rnd,mdl.tntf.tNtf_mdl.fnn.Fnn,mdl.tntf.tNtf_mdl.bnn.Bnn]"`




---



**Hyperparameters**

`OpeNTF` leverage [`hydra`](https://hydra.cc/) to manage models hyperparameters in hierarchy:

*   [`src/__config__.yaml`](https://github.com/fani-lab/OpeNTF/blob/main/src/__config__.yaml): `OpeNTF`'s main settings for the pipeline execution like `data.*`, `models.*`, `train.*`, `test.*`, `eval.*`
    *   [`src/mdl/__config__.yaml`](https://github.com/fani-lab/OpeNTF/blob/main/src/mdl/__config__.yaml): models' training hyperparameters like `fnn.*`, `bnn.*`, `lr`, `batch_size`, ...

To set these hyperparameters,

- `Override` them in the running commands (recommended), or
- Change the defaults in the `__config__.yaml` files


> An imortant config for the temporal training is `train.step_ahead` that must be set in to order the training set ascending in time and split the last `step_ahead` number of time steps as the `test` and all the previous time intervals for the per-time-interval training sets:

```
python main.py  "cmd=[prep,train,test,eval]" \
                "models.instances=[mdl.tntf.tNtf_mdl.fnn.Fnn,mdl.tntf.tNtf_mdl.bnn.Bnn]" \
                train.step_ahead=1 \
                data.domain=cmn.publication.Publication \
                data.source=../data/dblp/toy.dblp.v12.json \
                data.output=../output/dblp/toy.dblp.v12.json \
                ~data.filter \
                +models.batch_size=2 \
                +models.nsd=unigram_b \
                "+models.h=[128]"
```

(see default settings for common models in [`src/mdl/__config__.yaml`](https://github.com/fani-lab/OpeNTF/blob/main/src/mdl/__config__.yaml))




---



**Setup & Quickstart**

From the [`quickstart`](https://colab.research.google.com/github/fani-lab/OpeNTF/blob/main/ipynb/quickstart.ipynb) script:


In [1]:
# set up python 3.8
!sudo apt-get update -y
!sudo apt-get install -y python3.8 python3.8-venv python3.8-distutils python3-pip
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 10
!python --version

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.8 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,648 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [61.5 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,640 kB]
Hit:13 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:14 http:/

In [2]:
# get OpeNTF
!rm -R opentf/
!git clone https://github.com/Fani-Lab/opentf
!pip install --upgrade pip setuptools
!pip install -r opentf/requirements.txt

rm: cannot remove 'opentf/': No such file or directory
Cloning into 'opentf'...
remote: Enumerating objects: 27098, done.
remote: Counting objects: 100% (273/273), done.
remote: Compressing objects: 100% (208/208), done.
remote: Total 27098 (delta 121), reused 157 (delta 64), pack-reused 26825 (from 3)
Receiving objects: 100% (27098/27098), 1.32 GiB | 24.85 MiB/s, done.
Resolving deltas: 100% (13393/13393), done.
Updating files: 100% (4379/4379), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.3 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 68.1.2
    Not uninstalling setuptools at /usr/lib/python3/dist-packages, outside environment /usr
    Can't uninstall 'setuptools'. No files were found to uninstall.
  Attempting uninstall: pip
    Found existing installation: pip 22.0.2
    Not uninstalling pip at /usr/lib/python3/dist-packages, outside

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 150.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.0/806.0 kB 32.2 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144553 sha256=aceffbdf4ca2ac8938f274ecd60f3043a6223014ee72d6dcd2d8d0b6fce752c0
  Stored in directory: /root/.cache/pip/wheels/b1/a3/c2/6df046c09459b73cc9bb6c4401b0be6c47048baf9a1617c485
Successfully built antlr4-python3-runtime
  Attempting uninstall: zipp
    Found existing installation: zipp 1.0.0
    Uninstalling zipp-1.0.0:
      Successfully uninstalled zipp-1.0.0


In [3]:
%cd opentf/src/
!python main.py "cmd=[prep,train,test,eval]" "models.instances=[mdl.tntf.tNtf_mdl.bnn.Bnn]" train.step_ahead=1 data.domain=cmn.publication.Publication data.source=../data/dblp/toy.dblp.v12.json data.output=../output/dblp/toy.dblp.v12.json ~data.filter +models.batch_size=2 +models.nsd=unigram_b "+models.h=[128]"

/content/opentf/src
[2026-01-24 05:46:54,280][cmn.team][INFO] - Loading teamsvecs matrices from ../output/dblp/toy.dblp.v12.json/teamsvecs.pkl ...
[2026-01-24 05:46:54,281][pkgmgr][INFO] - tqdm not found.
[2026-01-24 05:46:54,281][pkgmgr][INFO] - Installing tqdm...
[2026-01-24 05:46:55,482][pkgmgr][INFO] - Collecting tqdm==4.65.0

[2026-01-24 05:46:55,487][cmn.team][INFO] - Loading indexes pickle from ../output/dblp/toy.dblp.v12.json/indexes.pkl ...
[2026-01-24 05:46:55,487][cmn.team][INFO] - Indexes pickle is loaded.
[2026-01-24 05:46:55,487][cmn.team][INFO] - Teamsvecs matrices and indexes for skills (31, 10), members (31, 13), and locations (31, 29) are loaded.
[2026-01-24 05:46:55,488][__main__][INFO] - Loading splits from ../output/dblp/toy.dblp.v12.json/splits.f3.r0.85.t1.pkl ...
[2026-01-24 05:46:55,488][pkgmgr][INFO] - sklearn.model_selection not found.
[2026-01-24 05:46:55,488][pkgmgr][INFO] - Installing scikit-learn...
[2026-01-24 05:47:00,562][pkgmgr][INFO] - Collecting scik

In [5]:
!ls ../output/dblp/toy.dblp.v12.json/splits.f3.r0.85.t1/bnn.b2.e100.ns5.lr0.001.es5.h[128].spe10.lbce.tpw10.tnw1.nsdunigram_b.nmc10

2000  2001  2002  2003	2004


In [6]:
!ls ../output/dblp/toy.dblp.v12.json/splits.f3.r0.85.t1/bnn.b2.e100.ns5.lr0.001.es5.h[128].spe10.lbce.tpw10.tnw1.nsdunigram_b.nmc10/2004

f0.e0.pt			   f1.test.e9.pred.eval.mean.csv
f0.e9.pt			   f1.test.pred
f0.pt				   f1.test.pred.eval.instance.csv
f0.test.e0.pred			   f1.test.pred.eval.mean.csv
f0.test.e0.pred.eval.instance.csv  f2.e0.pt
f0.test.e0.pred.eval.mean.csv	   f2.e9.pt
f0.test.e9.pred			   f2.pt
f0.test.e9.pred.eval.instance.csv  f2.test.e0.pred
f0.test.e9.pred.eval.mean.csv	   f2.test.e0.pred.eval.instance.csv
f0.test.pred			   f2.test.e0.pred.eval.mean.csv
f0.test.pred.eval.instance.csv	   f2.test.e9.pred
f0.test.pred.eval.mean.csv	   f2.test.e9.pred.eval.instance.csv
f1.e0.pt			   f2.test.e9.pred.eval.mean.csv
f1.e9.pt			   f2.test.pred
f1.pt				   f2.test.pred.eval.instance.csv
f1.test.e0.pred			   f2.test.pred.eval.mean.csv
f1.test.e0.pred.eval.instance.csv  logs4tboard
f1.test.e0.pred.eval.mean.csv	   splits.pkl
f1.test.e9.pred			   test.pred.eval.instance_mean.csv
f1.test.e9.pred.eval.instance.csv  test.pred.eval.mean.csv


In [7]:
import pandas as pd
pd.read_csv('/content/opentf/output/dblp/toy.dblp.v12.json/splits.f3.r0.85.t1/bnn.b2.e100.ns5.lr0.001.es5.h[128].spe10.lbce.tpw10.tnw1.nsdunigram_b.nmc10/2004/test.pred.eval.mean.csv', index_col = 0)


,mean,std
metrics,,
P_2,0.166667,0.144338
P_5,0.122222,0.038490
P_10,0.155556,0.050918
recall_2,0.166667,0.144338
recall_5,0.287037,0.089293
recall_10,0.722222,0.227371
ndcg_cut_2,0.154095,0.124005
ndcg_cut_5,0.222650,0.092011
ndcg_cut_10,0.398633,0.105452
